In [1]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
%matplotlib inline

In [2]:
nba = pd.read_csv('AllStarNBA')
nba_2016=pd.read_csv('NBA_2016')

In [3]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11933 entries, 0 to 11932
Data columns (total 53 columns):
Unnamed: 0      11933 non-null int64
player          11933 non-null object
pos             11933 non-null object
age             11933 non-null int64
bref_team_id    11933 non-null object
g               11933 non-null int64
gs              11933 non-null int64
mp              11933 non-null float64
fg              11933 non-null float64
fga             11933 non-null float64
fg_             11933 non-null float64
x3p             11933 non-null float64
x3pa            11933 non-null float64
x3p_            11933 non-null float64
x2p             11933 non-null float64
x2pa            11933 non-null float64
x2p_            11933 non-null float64
efg             11933 non-null float64
ft              11933 non-null float64
fta             11933 non-null float64
ft_             11933 non-null float64
orb             11933 non-null float64
drb             11933 non-null float64
trb  

In [48]:
nbaHistoric2= nba[nba.season_end <= 2015]
d1=nbaHistoric2
d_2016=nba_2016

In [5]:
d = nbaHistoric
d.info()

d_2015 = nba_2015
d_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10965 entries, 0 to 10964
Data columns (total 53 columns):
Unnamed: 0      10965 non-null int64
player          10965 non-null object
pos             10965 non-null object
age             10965 non-null int64
bref_team_id    10965 non-null object
g               10965 non-null int64
gs              10965 non-null int64
mp              10965 non-null float64
fg              10965 non-null float64
fga             10965 non-null float64
fg_             10965 non-null float64
x3p             10965 non-null float64
x3pa            10965 non-null float64
x3p_            10965 non-null float64
x2p             10965 non-null float64
x2pa            10965 non-null float64
x2p_            10965 non-null float64
efg             10965 non-null float64
ft              10965 non-null float64
fta             10965 non-null float64
ft_             10965 non-null float64
orb             10965 non-null float64
drb             10965 non-null float64
trb  

In [27]:
col_u = ["pts","ast","ft_","drb","blk","x2p","fg_","x3pa","tov","x3p_",'PER','WS',"ORB%",'WS/48','DWS','AST%',"TS%","TOV%","FTr","USG%"]

In [49]:
features= d[col_u]
features_2015 = d_2015[col_u]
#d.team.replace('None',0,inplace=True)
#labels=pd.to_numeric(d.team,errors='coerce')
features1= d1[col_u]
features_2016 = d_2016[col_u]


In [50]:
#d.team= np.where(d.team != 'None',1,0)
labels = d.team
labels1 = d1.team

In [30]:
labels.unique()

array(['None', '2.0', '3.0', '1.0'], dtype=object)

In [31]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=1)
train = pd.DataFrame(data=features_train, columns=[col_u])
train['team'] = labels_train
test = pd.DataFrame(data=features_test, columns=[col_u])
test['team'] = labels_test


# Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

clf = RandomForestClassifier(n_estimators=100,max_features='auto',min_samples_split=1)

clf = clf.fit(features_train,labels_train)

pred = clf.predict(features_test)

print "Accuracy =", accuracy_score(pred, labels_test)*100,"%"
scores = cross_val_score(clf, features_train, labels_train)
print scores.mean()
pd.DataFrame({'feature':col_u, 'importance':clf.feature_importances_}).sort_values(by='importance',ascending=False)

Accuracy = 98.2978723404 %
0.978371162759


,feature,importance
11,WS,0.111182
10,PER,0.103999
0,pts,0.102916
14,DWS,0.069971
5,x2p,0.056960
13,WS/48,0.055905
19,USG%,0.053000
3,drb,0.048173
8,tov,0.043496
6,fg_,0.037504


In [34]:
#features_2015 = d_2015[col_u]
d_2015['pred2015']=clf.predict(features_2015)
d_2015['rfprd']=clf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [36]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd
11202,LeBron James,1.0,2.0,0.32
11077,Stephen Curry,1.0,2.0,0.42
11082,Anthony Davis,1.0,2.0,0.37
11201,LeBron James,1.0,2.0,0.32
11131,Marc Gasol,1.0,None,0.08
11163,James Harden,1.0,1.0,0.13
10973,LaMarcus Aldridge,2.0,None,0.17
11328,Chris Paul,2.0,1.0,0.19
11437,Russell Westbrook,2.0,1.0,0.27
11132,Pau Gasol,2.0,None,0.22


# Adaboost

In [37]:

from sklearn.ensemble import AdaBoostClassifier
adaclf = AdaBoostClassifier(n_estimators=10)

adaclf = adaclf.fit(features_train,labels_train)

adapred = adaclf.predict(features_test)

print "Accuracy =", accuracy_score(adapred, labels_test)*100,"%"
adascores = cross_val_score(adaclf, features_train, labels_train)
print adascores.mean()
adaclf.feature_importances_

Accuracy = 97.3556231003 %
0.970032535257


array([ 0. ,  0. ,  0. ,  0. ,  0.1,  0. ,  0.1,  0. ,  0.1,  0. ,  0.4,
        0.3,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ])

In [38]:
#features_2015 = d_2015[col_u]
d_2015['adapredict']=adaclf.predict(features_2015)
d_2015['adaprb']=adaclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [39]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb
11202,LeBron James,1.0,2.0,0.32,None,0.224476
11077,Stephen Curry,1.0,2.0,0.42,1.0,0.255150
11082,Anthony Davis,1.0,2.0,0.37,None,0.258578
11201,LeBron James,1.0,2.0,0.32,None,0.224476
11131,Marc Gasol,1.0,None,0.08,None,0.239183
11163,James Harden,1.0,1.0,0.13,1.0,0.255150
10973,LaMarcus Aldridge,2.0,None,0.17,None,0.238493
11328,Chris Paul,2.0,1.0,0.19,None,0.258578
11437,Russell Westbrook,2.0,1.0,0.27,1.0,0.255150
11132,Pau Gasol,2.0,None,0.22,None,0.225023


# KNN

In [40]:
from sklearn.neighbors import KNeighborsClassifier
knnclf = KNeighborsClassifier(n_neighbors=20)

knnclf = knnclf.fit(features_train,labels_train)

knnpred = knnclf.predict(features_test)

print "Accuracy =", accuracy_score(knnpred, labels_test)*100,"%"
knnscores = cross_val_score(knnclf, features_train, labels_train)
print knnscores.mean()


Accuracy = 97.5075987842 %
0.971595849616


In [41]:
#features_2015 = d_2015[col_u]
d_2015['knnpred']=knnclf.predict(features_2015)
d_2015['knnprb']=knnclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [43]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb
11202,LeBron James,1.0,2.0,0.32,None,0.224476,1.0,0.30
11077,Stephen Curry,1.0,2.0,0.42,1.0,0.255150,1.0,0.30
11082,Anthony Davis,1.0,2.0,0.37,None,0.258578,1.0,0.20
11201,LeBron James,1.0,2.0,0.32,None,0.224476,1.0,0.30
11131,Marc Gasol,1.0,None,0.08,None,0.239183,None,0.05
11163,James Harden,1.0,1.0,0.13,1.0,0.255150,1.0,0.30
10973,LaMarcus Aldridge,2.0,None,0.17,None,0.238493,None,0.15
11328,Chris Paul,2.0,1.0,0.19,None,0.258578,2.0,0.40
11437,Russell Westbrook,2.0,1.0,0.27,1.0,0.255150,1.0,0.15
11132,Pau Gasol,2.0,None,0.22,None,0.225023,3.0,0.10


# Decision tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

dtclf = DecisionTreeClassifier(min_samples_split=1)

dtclf = dtclf.fit(features_train, labels_train)

dtpred = dtclf.predict(features_test)

print "accuracy =", dtclf.score(features_test, labels_test) * 100, '%'
dtscores = cross_val_score(dtclf, features_train, labels_train)
print dtscores.mean()
dtclf.feature_importances_

accuracy = 97.2948328267 %
0.968729433677


array([ 0.08154805,  0.03778536,  0.03975079,  0.03916301,  0.01780433,
        0.04184826,  0.04098429,  0.02174859,  0.01675822,  0.03067355,
        0.06760556,  0.24013351,  0.05547048,  0.04734853,  0.05985086,
        0.03341638,  0.02001943,  0.04123663,  0.04590726,  0.02094691])

In [45]:
#features_2015 = d_2015[col_u]
d_2015['dtpred']=dtclf.predict(features_2015)
d_2015['dtprb']=dtclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb','dtpred','dtprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [46]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb,dtpred,dtprb
11202,LeBron James,1.0,2.0,0.32,None,0.224476,1.0,0.30,2.0,1
11077,Stephen Curry,1.0,2.0,0.42,1.0,0.255150,1.0,0.30,1.0,0
11082,Anthony Davis,1.0,2.0,0.37,None,0.258578,1.0,0.20,None,0
11201,LeBron James,1.0,2.0,0.32,None,0.224476,1.0,0.30,2.0,1
11131,Marc Gasol,1.0,None,0.08,None,0.239183,None,0.05,None,0
11163,James Harden,1.0,1.0,0.13,1.0,0.255150,1.0,0.30,1.0,0
10973,LaMarcus Aldridge,2.0,None,0.17,None,0.238493,None,0.15,1.0,0
11328,Chris Paul,2.0,1.0,0.19,None,0.258578,2.0,0.40,None,0
11437,Russell Westbrook,2.0,1.0,0.27,1.0,0.255150,1.0,0.15,2.0,1
11132,Pau Gasol,2.0,None,0.22,None,0.225023,3.0,0.10,1.0,0


# Naive Bayes

In [27]:
from sklearn.naive_bayes import GaussianNB
nbclf = GaussianNB()

nbclf=nbclf.fit(features_train,labels_train)

nbpred=nbclf.predict(features_test)

print "accuracy =", nbclf.score(features_test, labels_test) * 100, '%'
nbscores = cross_val_score(nbclf, features_train, labels_train)
print nbscores.mean()


accuracy = 90.6990881459 %
0.899934737942


# SVC

In [28]:
from sklearn.svm import SVC
svcclf = SVC(kernel='rbf')

svcclf = svcclf.fit(features_train, labels_train)

svcpred = svcclf.predict(features_test)

print "accuracy =", svcclf.score(features_test, labels_test) * 100, '%'
svcscores = cross_val_score(svcclf, features_train, labels_train)
print svcscores.mean()


accuracy = 97.7811550152 %
0.97355034916


# 2016

In [51]:
X_train, X_test, y_train, y_test = train_test_split(features1, labels1, test_size=0.3, random_state=1)
train1 = pd.DataFrame(data=X_train, columns=[col_u])
train1['team'] = y_train
test1 = pd.DataFrame(data=X_test, columns=[col_u])
test1['team'] = y_test


In [52]:
from sklearn.tree import DecisionTreeClassifier

dtclf1 = DecisionTreeClassifier(min_samples_split=1)

dtclf1 = dtclf1.fit(X_train, y_train)

dtpred1 = dtclf1.predict(X_test)

print "accuracy =", dtclf1.score(X_test, y_test) * 100, '%'
dtscores1 = cross_val_score(dtclf1, X_train, y_train)
print dtscores1.mean()
dtclf1.feature_importances_

accuracy = 96.7713787086 %
0.971079054297


array([ 0.06293338,  0.02348972,  0.03151116,  0.02535715,  0.04417886,
        0.0574846 ,  0.01569536,  0.04135275,  0.0327307 ,  0.01776147,
        0.07136336,  0.26345815,  0.01481166,  0.06628026,  0.01866137,
        0.03092834,  0.05466055,  0.05275894,  0.02792453,  0.0466577 ])

In [64]:
#features_2015 = d_2015[col_u]
d_2016['dtpred1']=dtclf.predict(features_2016)
d_2016['dtprb1']=dtclf.predict_proba(features_2016)[:, 1]
#pred = clf.predict(features_test)


In [67]:
d_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 472
Data columns (total 53 columns):
Unnamed: 0      473 non-null int64
season          473 non-null int64
player          473 non-null object
pos             473 non-null object
age             473 non-null int64
bref_team_id    473 non-null object
g               473 non-null int64
gs              473 non-null int64
mp              473 non-null float64
fg              473 non-null float64
fga             473 non-null float64
fg_             473 non-null float64
x3p             473 non-null float64
x3pa            473 non-null float64
x3p_            473 non-null float64
x2p             473 non-null float64
x2pa            473 non-null float64
x2p_            473 non-null float64
efg             473 non-null float64
ft              473 non-null float64
fta             473 non-null float64
ft_             473 non-null float64
orb             473 non-null float64
drb             473 non-null float64
trb             473 